# Loading MIMIC data for downstream tasks and analysis in biomedical informatics, clinical research, and health AI

Steps:

1. Download https://physionet.org/content/mimiciii/1.4/ and https://physionet.org/content/mimiciv/2.2/ by creating a Physionet account, completing the training, and signing the data use agremeent (which includes provisions such as `The LICENSEE agrees to contribute code associated with publications arising from this data to a repository that is open to the research community.`)

2. Parse the header of each file and use the header information as context for a large language model to write the SQL scripts in DuckDB dialect. 
3. Make another LLM call and use the SQL query prototyped in this notebook as the basis for a dbt model, one per file.  

## Extract downloaded archives

In [2]:
!ls ~/data/mimic

mimic-iii-clinical-database-1.4.zip
mimic-iv-2.2.zip
mimic-iv-note-deidentified-free-text-clinical-notes-2.2.zip


In [3]:
!unzip ~/data/mimic/mimic-iii-clinical-database-1.4.zip -d ~/data/mimic

Archive:  /Users/me/data/mimic/mimic-iii-clinical-database-1.4.zip
 extracting: /Users/me/data/mimic/mimic-iii-clinical-database-1.4/ADMISSIONS.csv.gz  
 extracting: /Users/me/data/mimic/mimic-iii-clinical-database-1.4/CALLOUT.csv.gz  
 extracting: /Users/me/data/mimic/mimic-iii-clinical-database-1.4/CAREGIVERS.csv.gz  
 extracting: /Users/me/data/mimic/mimic-iii-clinical-database-1.4/CHARTEVENTS.csv.gz  
 extracting: /Users/me/data/mimic/mimic-iii-clinical-database-1.4/CPTEVENTS.csv.gz  
 extracting: /Users/me/data/mimic/mimic-iii-clinical-database-1.4/DATETIMEEVENTS.csv.gz  
 extracting: /Users/me/data/mimic/mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv.gz  
 extracting: /Users/me/data/mimic/mimic-iii-clinical-database-1.4/DRGCODES.csv.gz  
 extracting: /Users/me/data/mimic/mimic-iii-clinical-database-1.4/D_CPT.csv.gz  
 extracting: /Users/me/data/mimic/mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz  
 extracting: /Users/me/data/mimic/mimic-iii-clinical-database-1.4/D_ICD

In [5]:
!unzip ~/data/mimic/mimic-iv-2.2.zip -d ~/data/mimic

Archive:  /Users/me/data/mimic/mimic-iv-2.2.zip
  inflating: /Users/me/data/mimic/mimic-iv-2.2/CHANGELOG.txt  
  inflating: /Users/me/data/mimic/mimic-iv-2.2/LICENSE.txt  
  inflating: /Users/me/data/mimic/mimic-iv-2.2/SHA256SUMS.txt  
  inflating: /Users/me/data/mimic/mimic-iv-2.2/hosp/admissions.csv.gz  
  inflating: /Users/me/data/mimic/mimic-iv-2.2/hosp/d_hcpcs.csv.gz  
  inflating: /Users/me/data/mimic/mimic-iv-2.2/hosp/d_icd_diagnoses.csv.gz  
  inflating: /Users/me/data/mimic/mimic-iv-2.2/hosp/d_icd_procedures.csv.gz  
 extracting: /Users/me/data/mimic/mimic-iv-2.2/hosp/d_labitems.csv.gz  
  inflating: /Users/me/data/mimic/mimic-iv-2.2/hosp/diagnoses_icd.csv.gz  
  inflating: /Users/me/data/mimic/mimic-iv-2.2/hosp/drgcodes.csv.gz  
  inflating: /Users/me/data/mimic/mimic-iv-2.2/hosp/emar.csv.gz  
  inflating: /Users/me/data/mimic/mimic-iv-2.2/hosp/emar_detail.csv.gz  
  inflating: /Users/me/data/mimic/mimic-iv-2.2/hosp/hcpcsevents.csv.gz  
  inflating: /Users/me/data/mimic/mimic

In [1]:
!unzip ~/data/mimic/mimic-iv-note-deidentified-free-text-clinical-notes-2.2.zip -d ~/data/mimic

Archive:  /Users/me/data/mimic/mimic-iv-note-deidentified-free-text-clinical-notes-2.2.zip
replace /Users/me/data/mimic/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/LICENSE.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [7]:
%%capture folder_structure
!tree ~/data/mimic

# First load MIMIC-III data

In [1]:
%%capture mimic_iii_structure
import seedir as sd
mimic_iii_path = '~/data/mimic/mimic-iii-clinical-database-1.4'
sd.seedir(mimic_iii_path, style='lines', depthlimit=2, exclude_folders=['.git', '.ipynb_checkpoints'])

In [2]:
print(mimic_iii_structure.stdout)

mimic-iii-clinical-database-1.4/
├─DATETIMEEVENTS.csv.gz
├─CPTEVENTS.csv.gz
├─PATIENTS.csv.gz
├─DIAGNOSES_ICD.csv.gz
├─CAREGIVERS.csv.gz
├─PRESCRIPTIONS.csv.gz
├─INPUTEVENTS_MV.csv.gz
├─DRGCODES.csv.gz
├─D_ICD_DIAGNOSES.csv.gz
├─D_LABITEMS.csv.gz
├─TRANSFERS.csv.gz
├─ADMISSIONS.csv.gz
├─README.md
├─D_ITEMS.csv.gz
├─checksum_md5_zipped.txt
├─CALLOUT.csv.gz
├─D_CPT.csv.gz
├─LABEVENTS.csv.gz
├─PROCEDURES_ICD.csv.gz
├─CHARTEVENTS.csv.gz
├─SERVICES.csv.gz
├─LICENSE.txt
├─D_ICD_PROCEDURES.csv.gz
├─SHA256SUMS.txt
├─ICUSTAYS.csv.gz
├─INPUTEVENTS_CV.csv.gz
├─PROCEDUREEVENTS_MV.csv.gz
├─NOTEEVENTS.csv.gz
├─OUTPUTEVENTS.csv.gz
├─checksum_md5_unzipped.txt
└─MICROBIOLOGYEVENTS.csv.gz



In [3]:
%%capture code_folder_structure
sd.seedir('~/projects/electronic-health-records-analysis', style='lines', itemlimit=10, depthlimit=2, exclude_folders=['.git', '.ipynb_checkpoints'])

In [4]:
print(code_folder_structure.stdout)

electronic-health-records-analysis/
├─LICENSE
├─requirements.txt
├─data_processing/
│ ├─snapshots/
│ ├─tests/
│ ├─models/
│ ├─README.md
│ ├─macros/
│ ├─.gitignore
│ ├─seeds/
│ ├─analyses/
│ └─dbt_project.yml
├─README.md
├─logs/
│ └─dbt.log
├─.gitignore
├─.venv/
│ ├─bin/
│ ├─include/
│ ├─etc/
│ ├─pyvenv.cfg
│ ├─lib/
│ └─share/
├─notebooks/
│ └─loading_physionet_mimic_data.ipynb
└─requirements.in



## Get header of each file and use it as context for a large language model to write the SQL scripts in DuckDB dialect to load the MIMIC data into a database that can be saved to a parquet file

You may need to follow this readme first: https://colab.research.google.com/github/jaanli/language-model-notebooks/blob/main/notebooks/getting-started.ipynb

In [5]:
# Load duckdb, which lets us efficiently load large files
import duckdb

# Load pandas, which lets us manipulate dataframes
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True

%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Allow named parameters (python variables) in SQL cells
%config SqlMagic.named_parameters=True

# Connect jupysql to DuckDB using a SQLAlchemy-style connection string. Either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:

/Users/me/projects/electronic-health-records-analysis/.venv/lib/python3.9/site-packages/sql/traits.py:20: FutureWarning: named_parameters: boolean values are now deprecated. Value True will be treated as "enabled". 
Please use a valid option: "warn", "enabled", or "disabled". 
For more information, see the docs: https://jupysql.ploomber.io/en/latest/api/configuration.html#named-parameters
  warnings.warn(


In [6]:
%load_ext jupyter_ai

In [7]:
%load_ext dotenv

In [8]:
%dotenv

In [9]:
url = "https://raw.githubusercontent.com/MIT-LCP/mimic-code/main/mimic-iv/buildmimic/mysql/load.sql"

In [10]:
%%capture path_to_mimic

!readlink -f ~/data/mimic

In [11]:
path_to_mimic.stdout

'/Users/me/data/mimic\r\n'

In [12]:
%%capture example_contents

!curl {url}

In [13]:
%%capture example_name

!basename {url}

In [14]:
print(example_name)
print(example_contents.stdout[:100])

load.sql

-- csv2mysql with arguments:
--   -o
--   1-load-no-keys.sql
--   -e
--   
--   -u
--   -z
--


In [15]:
!pwd

/Users/me/projects/electronic-health-records-analysis/notebooks


In [16]:
!cd {mimic_iii_path}

In [17]:
import os
os.chdir(os.path.expanduser(mimic_iii_path))

In [18]:
!pwd

/Users/me/data/mimic/mimic-iii-clinical-database-1.4


In [19]:
%%capture row_level_protected_health_information
!for file in *.csv.gz; do echo "$file"; echo "----------"; duckdb -markdown -c "SELECT * FROM read_csv_auto('$file') LIMIT 10;"; echo "=========="; done

In [20]:
headers = row_level_protected_health_information.stdout.split('\n')
for i, row in enumerate(headers):
    if i > 0 and '.csv.gz' in headers[i - 1]:
        print(headers[i-1])
        print(headers[i+1])

ADMISSIONS.csv.gz
| ROW_ID | SUBJECT_ID | HADM_ID |      ADMITTIME      |      DISCHTIME      |      DEATHTIME      | ADMISSION_TYPE |    ADMISSION_LOCATION     |    DISCHARGE_LOCATION     | INSURANCE | LANGUAGE |     RELIGION      | MARITAL_STATUS |       ETHNICITY       |      EDREGTIME      |      EDOUTTIME      |                        DIAGNOSIS                         | HOSPITAL_EXPIRE_FLAG | HAS_CHARTEVENTS_DATA |
CALLOUT.csv.gz
| ROW_ID | SUBJECT_ID | HADM_ID | SUBMIT_WARDID | SUBMIT_CAREUNIT | CURR_WARDID | CURR_CAREUNIT | CALLOUT_WARDID | CALLOUT_SERVICE | REQUEST_TELE | REQUEST_RESP | REQUEST_CDIFF | REQUEST_MRSA | REQUEST_VRE | CALLOUT_STATUS | CALLOUT_OUTCOME | DISCHARGE_WARDID | ACKNOWLEDGE_STATUS |     CREATETIME      |     UPDATETIME      |   ACKNOWLEDGETIME   |     OUTCOMETIME     | FIRSTRESERVATIONTIME | CURRENTRESERVATIONTIME |
CAREGIVERS.csv.gz
| ROW_ID | CGID  | LABEL |      DESCRIPTION      |
CHARTEVENTS.csv.gz
| ROW_ID | SUBJECT_ID | HADM_ID | ICUSTAY_ID | ITEMID 

In [70]:
%%capture duckdb_docs_raw

!curl -s "https://duckdb.org/docs/data/csv/overview.html" | sed -e 's/<[^>]*>//g; /^$/d' | tr -s '\n'
!curl -s "https://duckdb.org/docs/sql/functions/dateformat.html" | sed -e 's/<[^>]*>//g; /^$/d' | tr -s '\n'

In [71]:
import re
duckdb_docs = re.escape(duckdb_docs_raw.stdout.replace('\t', '').replace('\n', '').replace(':', '').replace('{', '').replace('}', ''))

In [87]:
parent_prompt = f"""
Please find the context for this task here: 

```
{mimic_iii_structure.stdout}
```

This lists the directory contents using the seedir python package, of the directory at this path: `{mimic_iii_path}`.

Here are the headers of every file, derived from the command `for file in *.csv.gz; do echo "$file"; echo "----------"; duckdb -markdown -c "SELECT * FROM read_csv_auto('$file') LIMIT 10;"; echo "=========="; done`: 

```
{row_level_protected_health_information.stdout}
```

Additionally, here is an example file, {example_name} - in MYSQL, not DuckDB dialect - which contains a SQL transform to load the raw data from the files above into a database, using the following script:

```
{example_contents.stdout}
```

Remember:

We are running this notebook from the same directory as that shown above, from within the `notebooks` directory in Jupyter Lab. All of your code should be prefixed with the `%%sql` magic (the duckdb and jupysql libraries are already loaded!) so that it can be run after this first execution and code writing stage :) 

Also remember that this is the code folder structure: 

{code_folder_structure.stdout}

Also remember that these are the latest DuckDB documentation for the read_csv function that you must use: 

```
{duckdb_docs}
```

For each set of MIMIC files (and versions), get header of each file and use it as context for a large language model to write the SQL scripts in DuckDB dialect to load the MIMIC data into a database that can be saved to a parquet file.

Proceed step-by-step, focused on not changing the underlying names and original data dictionary at all. Make sure to cross-link data appropriately using the previously given file as reference.

Remember to always use the `read_csv` functions wrapped around the file name, like so: `SELECT * FROM read_csv('filename.csv.gz')` as but one example. 

Copy the resulting database into a parquet file compressed with ZSTD compression.

Please always remember to prefix the output with `%%sql` for the JupySQL cell magic :)
"""

In [88]:
print(parent_prompt[:1000])


Please find the context for this task here: 

```
mimic-iii-clinical-database-1.4/
├─DATETIMEEVENTS.csv.gz
├─CPTEVENTS.csv.gz
├─PATIENTS.csv.gz
├─DIAGNOSES_ICD.csv.gz
├─CAREGIVERS.csv.gz
├─PRESCRIPTIONS.csv.gz
├─INPUTEVENTS_MV.csv.gz
├─DRGCODES.csv.gz
├─D_ICD_DIAGNOSES.csv.gz
├─D_LABITEMS.csv.gz
├─TRANSFERS.csv.gz
├─ADMISSIONS.csv.gz
├─README.md
├─D_ITEMS.csv.gz
├─checksum_md5_zipped.txt
├─CALLOUT.csv.gz
├─D_CPT.csv.gz
├─LABEVENTS.csv.gz
├─PROCEDURES_ICD.csv.gz
├─CHARTEVENTS.csv.gz
├─SERVICES.csv.gz
├─LICENSE.txt
├─D_ICD_PROCEDURES.csv.gz
├─SHA256SUMS.txt
├─ICUSTAYS.csv.gz
├─INPUTEVENTS_CV.csv.gz
├─PROCEDUREEVENTS_MV.csv.gz
├─NOTEEVENTS.csv.gz
├─OUTPUTEVENTS.csv.gz
├─checksum_md5_unzipped.txt
└─MICROBIOLOGYEVENTS.csv.gz

```

This lists the directory contents using the seedir python package, of the directory at this path: `~/data/mimic/mimic-iii-clinical-database-1.4`.

Here are the headers of every file, derived from the command `for file in *.csv.gz; do echo "$file"; echo "---------

In [89]:
%%capture directory
!ls  -lh

In [90]:
print(directory)

total 12957144
-rw-r--r--@ 1 me  staff   2.4M Mar 19  2019 ADMISSIONS.csv.gz
-rw-r--r--@ 1 me  staff   1.1M Mar 19  2019 CALLOUT.csv.gz
-rw-r--r--@ 1 me  staff    48K Mar 19  2019 CAREGIVERS.csv.gz
-rw-r--r--@ 1 me  staff   4.0G Mar 19  2019 CHARTEVENTS.csv.gz
-rw-r--r--@ 1 me  staff   4.7M Mar 19  2019 CPTEVENTS.csv.gz
-rw-r--r--@ 1 me  staff    52M Mar 19  2019 DATETIMEEVENTS.csv.gz
-rw-r--r--@ 1 me  staff   4.5M Mar 19  2019 DIAGNOSES_ICD.csv.gz
-rw-r--r--@ 1 me  staff   1.7M Mar 19  2019 DRGCODES.csv.gz
-rw-r--r--@ 1 me  staff   3.9K Mar 19  2019 D_CPT.csv.gz
-rw-r--r--@ 1 me  staff   278K Mar 19  2019 D_ICD_DIAGNOSES.csv.gz
-rw-r--r--@ 1 me  staff    74K Mar 19  2019 D_ICD_PROCEDURES.csv.gz
-rw-r--r--@ 1 me  staff   184K Mar 19  2019 D_ITEMS.csv.gz
-rw-r--r--@ 1 me  staff    11K Mar 19  2019 D_LABITEMS.csv.gz
-rw-r--r--@ 1 me  staff   1.9M Mar 19  2019 ICUSTAYS.csv.gz
-rw-r--r--@ 1 me  staff   403M Mar 19  2019 INPUTEVENTS_CV.csv.gz
-rw-r--r--@ 1 me  staff   144M Mar 19  2019 INPU

## Blocker: need to copy and paste file names into the `%%ai` cell magic enabled by the `jupyter-ai` python package in order to execute the prompt separately for every file in the current working directory

In [91]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   2.4M Mar 19  2019 ADMISSIONS.csv.gz
```

In [94]:
%%sql

CREATE TABLE admissions AS
SELECT * FROM read_csv('ADMISSIONS.csv.gz', compression='gzip');

COPY admissions TO 'admissions.parquet' (FORMAT PARQUET, CODEC ZSTD);

,Success


In [95]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   1.1M Mar 19  2019 CALLOUT.csv.gz
```

In [96]:
%%sql

CREATE TABLE callout AS
SELECT *
FROM read_csv_auto('CALLOUT.csv.gz',
  delim=',',
  header=True,
  quote='"',
  escape='\',
  columns={
    row_id: 'INT',
    subject_id: 'INT',
    hadm_id: 'INT',
    submit_wardid: 'INT',
    submit_careunit: 'VARCHAR',
    curr_wardid: 'INT',
    curr_careunit: 'VARCHAR',
    callout_wardid: 'INT',
    callout_service: 'VARCHAR', 
    request_tele: 'INT',
    request_resp: 'INT',
    request_cdiff: 'INT',
    request_mrsa: 'INT',
    request_vre: 'INT',
    callout_status: 'VARCHAR',
    callout_outcome: 'VARCHAR',
    discharge_wardid: 'INT',
    acknowledge_status: 'VARCHAR',
    createtime: 'TIMESTAMP',
    updatetime: 'TIMESTAMP',
    acknowledgetime: 'TIMESTAMP',
    outcometime: 'TIMESTAMP',
    firstreservationtime: 'TIMESTAMP',
    currentreservationtime: 'TIMESTAMP'
  },
  dateformat='%Y-%m-%d %H:%M:%S',
  compression='gzip'
);

COPY callout TO 'callout.parquet' (FORMAT PARQUET, CODEC ZSTD);

,Success


In [97]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff    48K Mar 19  2019 CAREGIVERS.csv.gz
```

In [98]:
%%sql

CREATE TABLE CAREGIVERS AS 
SELECT * 
FROM read_csv('CAREGIVERS.csv.gz', header=True, compression='gzip');

COPY CAREGIVERS TO 'CAREGIVERS.parquet' (FORMAT 'parquet', CODEC 'ZSTD');

,Success


In [99]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   4.0G Mar 19  2019 CHARTEVENTS.csv.gz
```

In [100]:
%%sql

CREATE TABLE chartevents AS
SELECT *
FROM read_csv('CHARTEVENTS.csv.gz', header=True, columns={
    'ROW_ID': 'INT',
    'SUBJECT_ID': 'INT',
    'HADM_ID': 'INT',
    'ICUSTAY_ID': 'INT',
    'ITEMID': 'INT',
    'CHARTTIME': 'TIMESTAMP',
    'STORETIME': 'TIMESTAMP',
    'CGID': 'INT',
    'VALUE': 'VARCHAR',
    'VALUENUM': 'DOUBLE',
    'VALUEUOM': 'VARCHAR',
    'WARNING': 'BOOLEAN',
    'ERROR': 'BOOLEAN',
    'RESULTSTATUS': 'VARCHAR',
    'STOPPED': 'VARCHAR'
});

COPY chartevents TO 'CHARTEVENTS.parquet' (FORMAT PARQUET, CODEC ZSTD);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [101]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   4.7M Mar 19  2019 CPTEVENTS.csv.gz
```

In [102]:
%%sql

CREATE TABLE CPTEVENTS AS 
SELECT * FROM read_csv(
  'CPTEVENTS.csv.gz',
  delim=',',
  header=True,
  columns={
    'ROW_ID': 'INT',
    'SUBJECT_ID': 'INT',
    'HADM_ID': 'INT',
    'COSTCENTER': 'VARCHAR',
    'CHARTDATE': 'DATE',
    'CPT_CD': 'VARCHAR',
    'CPT_NUMBER': 'INT',
    'CPT_SUFFIX': 'VARCHAR', 
    'TICKET_ID_SEQ': 'INT',
    'SECTIONHEADER': 'VARCHAR',
    'SUBSECTIONHEADER': 'VARCHAR',
    'DESCRIPTION': 'VARCHAR'
  }
);

COPY (
  SELECT *
  FROM CPTEVENTS
) TO 'CPTEVENTS.parquet' (FORMAT PARQUET, CODEC ZSTD);

,Success


In [103]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff    52M Mar 19  2019 DATETIMEEVENTS.csv.gz
```

In [105]:
%%sql

CREATE TABLE DATETIMEEVENTS AS
SELECT *
FROM read_csv('DATETIMEEVENTS.csv.gz',
              compression='gzip');

COPY DATETIMEEVENTS TO 'DATETIMEEVENTS.parquet' (FORMAT PARQUET, CODEC ZSTD);

,Success


In [106]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   4.5M Mar 19  2019 DIAGNOSES_ICD.csv.gz
```

In [107]:
%%sql

CREATE TABLE diagnoses_icd AS
SELECT * FROM read_csv('DIAGNOSES_ICD.csv.gz', header=True, columns={
    'ROW_ID': 'INT',
    'SUBJECT_ID': 'INT',
    'HADM_ID': 'INT',
    'SEQ_NUM': 'INT',
    'ICD9_CODE': 'VARCHAR'
});

COPY (SELECT * FROM diagnoses_icd) TO 'diagnoses_icd.parquet' (FORMAT 'parquet', CODEC 'ZSTD');

,Success


In [108]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   1.7M Mar 19  2019 DRGCODES.csv.gz
```

In [109]:
%%sql

CREATE TABLE drgcodes AS
SELECT 
   CAST(subject_id AS INTEGER) AS subject_id,
   CAST(hadm_id AS INTEGER) AS hadm_id,
   CAST(drg_type AS VARCHAR) AS drg_type,
   CAST(drg_code AS VARCHAR) AS drg_code,
   CAST(description AS VARCHAR) AS description,
   CAST(drg_severity AS INTEGER) AS drg_severity,
   CAST(drg_mortality AS INTEGER) AS drg_mortality
FROM read_csv_auto('DRGCODES.csv.gz');

COPY drgcodes TO 'drgcodes.parquet' (FORMAT PARQUET, CODEC 'ZSTD');

,Success


In [110]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   3.9K Mar 19  2019 D_CPT.csv.gz
```

In [111]:
%%sql

CREATE TABLE D_CPT AS 
SELECT * FROM read_csv('D_CPT.csv.gz', header=True, compression='gzip');

COPY D_CPT TO 'D_CPT.parquet' (FORMAT PARQUET, CODEC ZSTD);

,Success


In [113]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   278K Mar 19  2019 D_ICD_DIAGNOSES.csv.gz
```

In [114]:
%%sql

CREATE TABLE D_ICD_DIAGNOSES AS
SELECT * FROM read_csv('D_ICD_DIAGNOSES.csv.gz', header=True, compression='gzip');

COPY D_ICD_DIAGNOSES TO 'D_ICD_DIAGNOSES.parquet' (FORMAT 'parquet', CODEC 'ZSTD');

,Success


In [ ]:
%%sql

CREATE TABLE d_icd_diagnoses AS 
SELECT *
FROM read_csv('D_ICD_DIAGNOSES.csv.gz', header=True, compression='gzip', delim=',', columns={
    'ROW_ID': 'INT',
    'ICD9_CODE': 'VARCHAR',
    'SHORT_TITLE': 'VARCHAR', 
    'LONG_TITLE': 'VARCHAR'
});

COPY d_icd_diagnoses TO 'D_ICD_DIAGNOSES.parquet' (FORMAT 'parquet', CODEC 'ZSTD');

In [115]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff    74K Mar 19  2019 D_ICD_PROCEDURES.csv.gz
```

In [116]:
%%sql

CREATE TABLE D_ICD_PROCEDURES AS
SELECT
    CAST(ROW_ID AS INTEGER) AS ROW_ID,
    CAST(ICD9_CODE AS VARCHAR) AS ICD9_CODE,
    CAST(SHORT_TITLE AS VARCHAR) AS SHORT_TITLE,
    CAST(LONG_TITLE AS VARCHAR) AS LONG_TITLE
FROM read_csv_auto('D_ICD_PROCEDURES.csv.gz');

COPY (SELECT * FROM D_ICD_PROCEDURES) TO 'D_ICD_PROCEDURES.parquet' (FORMAT 'parquet', CODEC 'ZSTD');

,Success


In [117]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   184K Mar 19  2019 D_ITEMS.csv.gz
```

In [120]:
%%sql
CREATE TABLE D_ITEMS AS 
SELECT *
FROM read_csv('D_ITEMS.csv.gz', header=True, delim=',', quote='"', 
  columns={
    'row_id': 'INT',
    'itemid': 'INT',
    'label': 'VARCHAR',
    'abbreviation': 'VARCHAR',
    'dbsource': 'VARCHAR',
    'linksto': 'VARCHAR',
    'category': 'VARCHAR',
    'unitname': 'VARCHAR',
    'param_type': 'VARCHAR',
    'conceptid': 'INT'
});

COPY D_ITEMS TO 'D_ITEMS.parquet' (FORMAT PARQUET, CODEC ZSTD);

,Success


In [118]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff    11K Mar 19  2019 D_LABITEMS.csv.gz
```

In [121]:
%%sql

CREATE TABLE D_LABITEMS AS 
SELECT * 
FROM read_csv('D_LABITEMS.csv.gz', header=True, compression='gzip');

COPY D_LABITEMS TO 'D_LABITEMS.parquet' (FORMAT PARQUET, CODEC ZSTD);

,Success


In [119]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   1.9M Mar 19  2019 ICUSTAYS.csv.gz
```

In [122]:
%%sql

CREATE TABLE icustays AS
SELECT * FROM read_csv('ICUSTAYS.csv.gz', 
    header=True,
    columns={
        'ROW_ID': 'INT',
        'SUBJECT_ID': 'INT',
        'HADM_ID': 'INT',
        'ICUSTAY_ID': 'INT',
        'DBSOURCE': 'VARCHAR',
        'FIRST_CAREUNIT': 'VARCHAR',
        'LAST_CAREUNIT': 'VARCHAR',
        'FIRST_WARDID': 'INT',
        'LAST_WARDID': 'INT',
        'INTIME': 'TIMESTAMP',
        'OUTTIME': 'TIMESTAMP',
        'LOS': 'DOUBLE'
    },
    compression='gzip'
);

COPY icustays TO 'icustays.parquet' (FORMAT 'parquet', CODEC 'ZSTD');

,Success


In [123]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   403M Mar 19  2019 INPUTEVENTS_CV.csv.gz
```

In [135]:
%%sql

CREATE TABLE inputevents_cv AS
SELECT *
FROM read_csv('INPUTEVENTS_CV.csv.gz', header=True, compression='gzip',
    columns={'ROW_ID': 'INT',
             'SUBJECT_ID': 'INT',
             'HADM_ID': 'INT',
             'ICUSTAY_ID': 'INT',
             'CHARTTIME': 'TIMESTAMP',
             'ITEMID': 'INT',
             'AMOUNT': 'DOUBLE',
             'AMOUNTUOM': 'VARCHAR',
             'RATE': 'DOUBLE',
             'RATEUOM': 'VARCHAR',
             'STORETIME': 'TIMESTAMP',
             'CGID': 'INT',
             'ORDERID': 'INT',
             'LINKORDERID': 'INT',
             'STOPPED': 'VARCHAR',
             'NEWBOTTLE': 'INT',
             'ORIGINALAMOUNT': 'DOUBLE',
             'ORIGINALAMOUNTUOM': 'VARCHAR',
             'ORIGINALROUTE': 'VARCHAR',
             'ORIGINALRATE': 'DOUBLE',
             'ORIGINALRATEUOM': 'VARCHAR',
             'ORIGINALSITE': 'VARCHAR'
            });

COPY inputevents_cv TO 'inputevents_cv.parquet' (FORMAT 'parquet', CODEC 'ZSTD');

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [124]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   144M Mar 19  2019 INPUTEVENTS_MV.csv.gz
```

In [138]:
%%sql

CREATE TABLE inputevents AS 
SELECT 
    *
FROM read_csv('INPUTEVENTS_MV.csv.gz');

COPY (SELECT * FROM inputevents) TO 'inputevents.parquet' (FORMAT PARQUET, CODEC ZSTD);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [125]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   320M Mar 19  2019 LABEVENTS.csv.gz
```

In [141]:
%%sql

CREATE TABLE labevents AS
SELECT
    *
FROM read_csv('LABEVENTS.csv.gz', delim=',', header=True, quote='"');

COPY labevents TO 'labevents.parquet' (FORMAT PARQUET, CODEC ZSTD);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [126]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   7.3M Mar 19  2019 MICROBIOLOGYEVENTS.csv.gz
```

In [145]:
%%sql

CREATE TABLE microbiologyevents AS
SELECT * FROM read_csv('MICROBIOLOGYEVENTS.csv.gz');

COPY (SELECT * FROM microbiologyevents) TO 'microbiologyevents.parquet' (FORMAT PARQUET, CODEC ZSTD);

,Success


In [158]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   1.1G Mar 19  2019 NOTEEVENTS.csv.gz
```

In [161]:
%%sql
CREATE OR REPLACE TABLE noteevents AS 
SELECT 
   CAST(row_id AS INT) AS row_id,
   CAST(subject_id AS INT) AS subject_id,
   CAST(hadm_id AS INT) AS hadm_id,
   CAST(chartdate AS DATE) AS chartdate,
   CAST(charttime AS TIME) AS charttime,
   CAST(storetime AS TIMESTAMP) AS storetime,
   CAST(category AS VARCHAR) AS category,
   CAST(description AS VARCHAR) AS description,
   CAST(cgid AS INT) AS cgid,
   CAST(iserror AS BOOLEAN) AS iserror,
   CAST(text AS VARCHAR) AS text
FROM read_csv('NOTEEVENTS.csv.gz', header=True, nullstr='', ignore_errors=true);

COPY noteevents TO 'noteevents.parquet' (FORMAT 'parquet', CODEC 'ZSTD');

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [128]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff    56M Mar 19  2019 OUTPUTEVENTS.csv.gz
```

In [150]:
%%sql

CREATE TABLE OUTPUTEVENTS AS (
  SELECT 
    CAST(subject_id AS INT) AS subject_id,
    CAST(hadm_id AS INT) AS hadm_id,
    CAST(icustay_id AS INT) AS icustay_id,
    CAST(cgid AS INT) AS cgid,
    CAST(charttime AS TIMESTAMP) AS charttime,
    CAST(storetime AS TIMESTAMP) AS storetime,
    CAST(itemid AS INT) AS itemid,
    CAST(value AS DOUBLE) AS value,
    CAST(valueuom AS VARCHAR) AS valueuom
  FROM read_csv_auto('OUTPUTEVENTS.csv.gz')
);

COPY OUTPUTEVENTS TO 'OUTPUTEVENTS.parquet' (FORMAT PARQUET, CODEC 'ZSTD');

,Success


In [129]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   558K Mar 19  2019 PATIENTS.csv.gz
```

In [151]:
%%sql

CREATE TABLE patients AS
SELECT *
FROM read_csv('PATIENTS.csv.gz', columns={
    'ROW_ID': 'INT',
    'SUBJECT_ID': 'INT',
    'GENDER': 'VARCHAR',
    'DOB': 'DATE',
    'DOD': 'DATE',
    'DOD_HOSP': 'DATE',
    'DOD_SSN': 'DATE',
    'EXPIRE_FLAG': 'INT'
}, ALL_VARCHAR=False, header=True);

COPY (SELECT * FROM patients) TO 'patients.parquet' (FORMAT PARQUET, CODEC ZSTD);

,Success


In [130]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff    99M Mar 19  2019 PRESCRIPTIONS.csv.gz
```

In [152]:
%%sql

CREATE TABLE PRESCRIPTIONS AS 
SELECT * FROM read_csv('PRESCRIPTIONS.csv.gz', 
    header=True,
    quote='"',
    escape='"',
    auto_detect=True
);

COPY PRESCRIPTIONS TO 'prescriptions.parquet' (FORMAT PARQUET, CODEC ZSTD);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [131]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   7.5M Mar 19  2019 PROCEDUREEVENTS_MV.csv.gz
```

In [154]:
%%sql

CREATE TABLE procedureevents_mv AS
SELECT 
    *
FROM read_csv('PROCEDUREEVENTS_MV.csv.gz');

COPY procedureevents_mv TO 'procedureevents_mv.parquet' (FORMAT PARQUET, CODEC ZSTD);

,Success


In [132]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   1.7M Mar 19  2019 PROCEDURES_ICD.csv.gz
```

In [155]:
%%sql

CREATE TABLE PROCEDURES_ICD AS 
SELECT * 
FROM read_csv('PROCEDURES_ICD.csv.gz', header=True, compression='gzip');

COPY PROCEDURES_ICD TO 'PROCEDURES_ICD.parquet' (FORMAT PARQUET, CODEC 'ZSTD');

,Success


In [133]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   1.1M Mar 19  2019 SERVICES.csv.gz
```

In [156]:
%%sql

CREATE TABLE SERVICES AS
SELECT *
FROM read_csv('SERVICES.csv.gz', compression='gzip', header=True, 
    columns={
        'ROW_ID': 'INT',
        'SUBJECT_ID': 'INT',
        'HADM_ID': 'INT',
        'TRANSFERTIME': 'TIMESTAMP',
        'PREV_SERVICE': 'VARCHAR',
        'CURR_SERVICE': 'VARCHAR'
    });

COPY (
    SELECT *
    FROM SERVICES
) TO 'SERVICES.parquet' (FORMAT 'parquet', CODEC 'ZSTD');

,Success


In [134]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file:

```
-rw-r--r--@ 1 me  staff   5.2M Mar 19  2019 TRANSFERS.csv.gz
```

In [157]:
%%sql

CREATE TABLE transfers AS
SELECT * FROM read_csv_auto('TRANSFERS.csv.gz');

COPY transfers TO 'transfers.parquet' (FORMAT PARQUET, CODEC 'ZSTD');

,Success


## Generate the schema based on the DuckDB database files

In [165]:
%%capture filenames
!ls -lh *.parquet

In [169]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Generate the schema using bash one liners in duckdb for every file here:
```
{filenames}                                                   
```

Next we will figure out how to link these files with join statements.

Update: Instead of `%%sql`, the prefix for this one cell containing the bash one liner will simply be `!` to indicate shell command.

In [171]:
%%capture mimic_iii_schema
!for file in *.parquet; do echo "$file"; echo "----------"; duckdb -c "DESCRIBE SELECT * FROM parquet_scan('$file')"; echo "=========="; done

In [172]:
print(mimic_iii_schema.stdout)

CAREGIVERS.parquet
----------
┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ROW_ID      │ BIGINT      │ YES     │         │         │         │
│ CGID        │ BIGINT      │ YES     │         │         │         │
│ LABEL       │ VARCHAR     │ YES     │         │         │         │
│ DESCRIPTION │ VARCHAR     │ YES     │         │         │         │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘
CHARTEVENTS.parquet
----------
┌──────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name  │ column_type │  null   │   key   │ default │  extra  │
│   varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ROW_ID       │ INTEGER 

## Generate visualizations of the MIMIC III database / clinical data repository using python and altair

In [185]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

Given the schema in these files: 

```
{mimic_iii_schema.stdout}
```

Proceed to create simple interactive figures using the vega-altair library in python. Use best practices for the visual display of information, such as those from Tufte.

Don't forget to load the parquet file first to ensure the python variables are available for altair :) and proceed step-by-step.

Your work output will be used for downstream analytics such as cohort selection and clinical trials paptient population matching and a *variety* of other use cases in biomedical informatics and life sciences.

Also remember to avoid this error: 

```
MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).

Try enabling the VegaFusion data transformer which raises this limit by pre-evaluating data
transformations in Python.
    >> import altair as alt
    >> alt.data_transformers.enable("vegafusion")
```

Only use the following variables:

```
ADMISSION_TYPE
GENDER
LOS
```

Alongside their human readable descriptions in the axes labels.

Bucket the length of stay into entire days; perhaps a histogram will suffice or some other standard way of showing such a business intelligence dashboard.

Alongside any others you might need to link the data in MIMIC-III :)

In [186]:
import altair as alt
import pandas as pd

# Enable VegaFusion
alt.data_transformers.enable("vegafusion")

# Load the parquet files into DataFrames
df_admissions = pd.read_parquet('admissions.parquet')
df_patients = pd.read_parquet('patients.parquet')
df_icustays = pd.read_parquet('icustays.parquet')

# Join the DataFrames
df = pd.merge(df_admissions, df_patients, on='SUBJECT_ID')
df = pd.merge(df, df_icustays, on=['SUBJECT_ID', 'HADM_ID'])

# Create a histogram of length of stay
los_hist = alt.Chart(df).mark_bar().encode(
    alt.X("LOS:Q", bin=True, title='Length of Stay (days)'),
    y='count()',
    color=alt.Color('ADMISSION_TYPE:N', title='Admission Type'),
    tooltip=['count()']
).properties(
    width=600,
    height=400,
    title='Distribution of Length of Stay by Admission Type'
)

# Create a bar chart of admission type by gender
adm_gender_bar = alt.Chart(df).mark_bar().encode(
    x=alt.X('ADMISSION_TYPE:N', title='Admission Type'),
    y=alt.Y('count()', title='Count'),
    color=alt.Color('GENDER:N', title='Gender'),
    tooltip=['count()']
).properties(
    width=400,
    height=300,
    title='Admission Type by Gender'
)

# Combine the charts
dashboard = alt.vconcat(los_hist, adm_gender_bar)

# Display the dashboard
dashboard

alt.VConcatChart(...)